In [288]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [289]:
align = pd.read_csv('alignment.csv')
blacla = pd.read_csv('blastclass.csv')
blaptn = pd.read_csv('blastpattern.csv')
obstr = pd.read_csv('obstruction.csv')
ppvkn = pd.read_csv('ppv.csv')
rockcla = pd.read_csv('rockclass.csv')
eqn_no = 1


In [290]:
# print(align,'\n\n')
print(blacla,'\n\n')
# print(blaptn,'\n\n')
# print(obs,'\n\n')
# print(ppv,'\n\n')
# print(rockcla,'\n\n')

   sta  class
0   10      7
1   20      5
2   30      3
3   40      3
4   50      2
5   60      1
6   70      3
7   80      4
8   90      5
9  100      6 




In [291]:
def ppv(K,n,weight=1,distance=1): 
    sd = distance/math.sqrt(weight)
    v = K*math.pow(sd,n)
    return v

In [292]:
K = ppvkn['K'][eqn_no]
n = ppvkn['n'][eqn_no]
W = blaptn['exp_weight'][0]

print(f'K = {K}, n = {n}, W = {W}')
for d in range(1,10):
    print(f'ppv of K ={K} n ={n} W ={W} & d ={d} = {ppv(K,n,W,d)}')

K = 140, n = -1.2, W = 2.5
ppv of K =140 n =-1.2 W =2.5 & d =1 = 242.60069510430122
ppv of K =140 n =-1.2 W =2.5 & d =2 = 105.59808588954034
ppv of K =140 n =-1.2 W =2.5 & d =3 = 64.91522029071477
ppv of K =140 n =-1.2 W =2.5 & d =4 = 45.96423657706593
ppv of K =140 n =-1.2 W =2.5 & d =5 = 35.16641004113413
ppv of K =140 n =-1.2 W =2.5 & d =6 = 28.25599079528687
ppv of K =140 n =-1.2 W =2.5 & d =7 = 23.484125514459596
ppv of K =140 n =-1.2 W =2.5 & d =8 = 20.00709602182053
ppv of K =140 n =-1.2 W =2.5 & d =9 = 17.3700484393926


In [293]:
algn = np.array(align)
print(algn)

class Align:
    def __init__(self,align):
        self.align = align
        self.Sta = align['sta']
        self.X = align['x']
        self.Y = align['y']
        self.Z = align['z']
    
    def get(self,sta):
        index = self.ind(sta)

        t = (sta - self.Sta[index])/(self.Sta[index+1] - self.Sta[index])
        x = self.X[index] + t*(self.X[index+1] - self.X[index])
        y = self.Y[index] + t*(self.Y[index+1] - self.Y[index])
        z = self.Z[index] + t*(self.Z[index+1] - self.Z[index])
        return (x,y,z)

    def ind(self,sta):
        if sta < self.Sta[0]:
            return self.Sta[0]
        elif sta > self.Sta[len(self.Sta)-1]:
            return self.Sta[len(self.Sta)-1]
        index = self.Sta[0]
        for i in range(len(self.Sta)-1):
            if self.Sta[i] <= sta and sta < self.Sta[i+1]:
                index = i
                break
        return index

    
    


algn = Align(align)
print(algn.get(3.5))


[[  0.    0.   10.  -10. ]
 [ 10.   10.    8.  -20. ]
 [ 20.   20.    6.  -30. ]
 [ 30.   30.    4.  -40.5]
 [ 40.   40.    4.  -50. ]
 [ 50.   50.    2.  -69.5]
 [ 60.   60.    2.  -69.5]
 [ 70.   70.    4.  -59.5]
 [ 80.   80.    6.  -59.5]
 [ 90.   90.    8.  -40. ]
 [100.  100.   10.  -20. ]]
(3.5, 9.3, -13.5)


In [294]:
K = ppvkn['K'][0]
n = ppvkn['n'][0]
cla = 2
W = blaptn[blaptn['class']==cla].values[0][2] #blast pattern design
print(f'K = {K}, n = {n}, W = {W}')

sta = 20
print(blacla[blacla['sta']==sta]['class'])
cla = blacla[blacla['sta']==sta]['class'].values[0] #blast class along the alignment
print(f'cla = {cla}')
W = blaptn[blaptn['class']==cla].values[0][2] #blast pattern design
print(f'K = {K}, n = {n}, W = {W}')

obs = np.array(obstr)

obs_allow = obs[:,4]
obs_pos = [(obs[i,1],obs[i,2],obs[i,3]) for i in range(len(obs))]
print(obs_allow)
print(obs_pos)
for i,sta in enumerate(algn.Sta[:-1]):
    print(f'i = {i}, sta = {sta}')
    bc = blacla['class'][i]
    W = blaptn[blaptn['class']==bc].values[0][2] #blast pattern design
    print(f'sta = {sta},W = {W}')
    for op,alw in zip(obs_pos,obs_allow):
        apos = algn.get(sta)
        dis = math.sqrt((op[0]-apos[0])**2 + (op[1]-apos[1])**2 + (op[2]-apos[2])**2)
        pvel = ppv(K,n,W,dis)
        print(f'op = {op},apos = {apos} dis = {dis} pvel = {pvel} alw = {alw} {"OK" if pvel <= alw else "NG****"}')


K = 200, n = -1.6, W = 1.0
1    5
Name: class, dtype: int64
cla = 5
K = 200, n = -1.6, W = 0.5
[1.  0.5]
[(30.0, 10.0, 1.0), (50.0, 5.0, 1.0)]
i = 0, sta = 0
sta = 0,W = 0.1
op = (30.0, 10.0, 1.0),apos = (0.0, 10.0, -10.0) dis = 31.953090617340916 pvel = 0.12411075055275028 alw = 1.0 OK
op = (50.0, 5.0, 1.0),apos = (0.0, 10.0, -10.0) dis = 51.43928459844674 pvel = 0.05793728285465125 alw = 0.5 OK
i = 1, sta = 10
sta = 10,W = 0.5
op = (30.0, 10.0, 1.0),apos = (10.0, 8.0, -20.0) dis = 29.068883707497267 pvel = 0.5232637291457816 alw = 1.0 OK
op = (50.0, 5.0, 1.0),apos = (10.0, 8.0, -20.0) dis = 45.27692569068709 pvel = 0.2575152246927217 alw = 0.5 OK
i = 2, sta = 20
sta = 20,W = 0.8
op = (30.0, 10.0, 1.0),apos = (20.0, 6.0, -30.0) dis = 32.817678162843876 pvel = 0.6276671830888257 alw = 1.0 OK
op = (50.0, 5.0, 1.0),apos = (20.0, 6.0, -30.0) dis = 43.15089802078283 pvel = 0.40505896058795077 alw = 0.5 OK
i = 3, sta = 30
sta = 30,W = 0.8
op = (30.0, 10.0, 1.0),apos = (30.0, 4.0, -40.5) dis